# Notes on Bitcoin Whitepaper (Page 1)

## The Problem with Traditional Online Payments

*   **Dependence on Trust:** All online commerce relies on trusted third-party financial institutions (like banks or PayPal) to process payments.
*   **Reversibility Issues:** This trust-based model allows for transaction reversals (chargebacks). This creates higher transaction costs as merchants must protect themselves from fraud.
*   **Inherent Weakness:** The system is fundamentally based on trust, not on mathematical proof, which limits its ability to handle certain types of transactions and protect against all fraud.

## The Proposed Solution: A Peer-to-Peer Electronic Cash System

The paper proposes a system for electronic transactions that does not rely on trust. Instead, it uses **cryptographic proof**.

In [1]:
# Let's implement a simple demonstration of cryptographic proof vs trust
import hashlib
import time
import random

class SimpleBitcoinDemo:
    def __init__(self):
        self.transactions = []
        self.blocks = []
        
    def create_transaction(self, sender, receiver, amount):
        """Create a new transaction"""
        tx = {
            'sender': sender,
            'receiver': receiver,
            'amount': amount,
            'timestamp': time.time()
        }
        return tx
    
    def hash_transaction(self, transaction):
        """Create a hash of a transaction"""
        tx_string = f"{transaction['sender']}{transaction['receiver']}{transaction['amount']}{transaction['timestamp']}"
        return hashlib.sha256(tx_string.encode()).hexdigest()
    
    def demonstrate_cryptographic_proof(self):
        """Show how cryptographic proof works"""
        print("🔐 Cryptographic Proof Demonstration")
        print("=" * 50)
        
        # Create a transaction
        tx = self.create_transaction("Alice", "Bob", 5.0)
        tx_hash = self.hash_transaction(tx)
        
        print(f"Transaction: {tx}")
        print(f"Transaction Hash: {tx_hash}")
        print()
        
        # Show that even tiny changes create completely different hashes
        print("📝 Demonstrating Hash Sensitivity:")
        modified_tx = tx.copy()
        modified_tx['amount'] = 5.1  # Change amount by 0.1
        modified_hash = self.hash_transaction(modified_tx)
        
        print(f"Original Amount: {tx['amount']} → Hash: {tx_hash[:16]}...")
        print(f"Modified Amount: {modified_tx['amount']} → Hash: {modified_hash[:16]}...")
        print(f"Hashes are {'identical' if tx_hash == modified_hash else 'completely different'}")
        
        return tx_hash

# Run the demonstration
demo = SimpleBitcoinDemo()
demo.demonstrate_cryptographic_proof()

🔐 Cryptographic Proof Demonstration
Transaction: {'sender': 'Alice', 'receiver': 'Bob', 'amount': 5.0, 'timestamp': 1751820246.154523}
Transaction Hash: 0491e2e62eaed86019783316a048860bdef32e4d0329fa5d52283547a35d32ed

📝 Demonstrating Hash Sensitivity:
Original Amount: 5.0 → Hash: 0491e2e62eaed860...
Modified Amount: 5.1 → Hash: 4ebb10c098b7b560...
Hashes are completely different


'0491e2e62eaed86019783316a048860bdef32e4d0329fa5d52283547a35d32ed'

In [2]:
## How It Works

1.  **Public Transactions:** All transactions are announced publicly to a network of peers.
2.  **Proof-of-Work:** To validate and timestamp transactions, the network uses a Proof-of-Work system. Participants (miners) must solve a difficult computational puzzle. This is what the `assignment3.js` script simulates.
3.  **The Blockchain:** The solution to the puzzle (the proof-of-work) is added to a block of transactions, which is then cryptographically linked to the previous block. This forms a secure chain that is very difficult to change.
4.  **Security through Majority Power:** The longest chain serves as the official record. As long as the majority of the network's computing power is controlled by honest nodes, they will consistently outpace any attackers, making the network secure.

In [3]:
# Interactive Blockchain Simulation
class SimpleBlockchain:
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        self.mining_difficulty = 4  # Number of leading zeros required
        
    def create_genesis_block(self):
        """Create the first block in the chain"""
        genesis_block = {
            'index': 0,
            'timestamp': time.time(),
            'transactions': ['Genesis Block'],
            'previous_hash': '0',
            'nonce': 0,
            'hash': None
        }
        genesis_block['hash'] = self.calculate_hash(genesis_block)
        self.chain.append(genesis_block)
        return genesis_block
    
    def calculate_hash(self, block):
        """Calculate the hash of a block"""
        block_string = f"{block['index']}{block['timestamp']}{block['transactions']}{block['previous_hash']}{block['nonce']}"
        return hashlib.sha256(block_string.encode()).hexdigest()
    
    def mine_block(self, transactions):
        """Mine a new block using Proof of Work"""
        print(f"⛏️  Mining new block with {len(transactions)} transactions...")
        
        new_block = {
            'index': len(self.chain),
            'timestamp': time.time(),
            'transactions': transactions,
            'previous_hash': self.chain[-1]['hash'] if self.chain else '0',
            'nonce': 0,
            'hash': None
        }
        
        # Proof of Work: find a hash that starts with required number of zeros
        start_time = time.time()
        while True:
            hash_attempt = self.calculate_hash(new_block)
            if hash_attempt.startswith('0' * self.mining_difficulty):
                new_block['hash'] = hash_attempt
                mining_time = time.time() - start_time
                print(f"✅ Block mined! Nonce: {new_block['nonce']}, Time: {mining_time:.2f}s")
                print(f"   Hash: {hash_attempt}")
                break
            new_block['nonce'] += 1
            
            # Safety check to prevent infinite loops in demo
            if new_block['nonce'] > 1000000:
                print("Mining taking too long, reducing difficulty...")
                self.mining_difficulty = max(1, self.mining_difficulty - 1)
                new_block['nonce'] = 0
        
        self.chain.append(new_block)
        return new_block
    
    def validate_chain(self):
        """Validate the entire blockchain"""
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            
            # Check if current block's hash is valid
            if current_block['hash'] != self.calculate_hash(current_block):
                return False
            
            # Check if current block points to previous block
            if current_block['previous_hash'] != previous_block['hash']:
                return False
                
        return True
    
    def display_chain(self):
        """Display the blockchain in a readable format"""
        print("\n📊 Blockchain Status:")
        print("=" * 60)
        for block in self.chain:
            print(f"Block #{block['index']}:")
            print(f"  Timestamp: {time.ctime(block['timestamp'])}")
            print(f"  Transactions: {block['transactions']}")
            print(f"  Previous Hash: {block['previous_hash'][:16]}...")
            print(f"  Nonce: {block['nonce']}")
            print(f"  Hash: {block['hash'][:16]}...")
            print(f"  Valid: ✅" if self.validate_chain() else "❌")
            print("-" * 40)

# Create and demonstrate a simple blockchain
print("🔗 Creating a Simple Blockchain")
blockchain = SimpleBlockchain()

# Create genesis block
genesis = blockchain.create_genesis_block()
print(f"Genesis block created with hash: {genesis['hash'][:16]}...")

# Mine some blocks
blockchain.mine_block(['Alice sends 5 BTC to Bob'])
blockchain.mine_block(['Bob sends 2 BTC to Charlie', 'Alice sends 3 BTC to David'])
blockchain.mine_block(['Charlie sends 1 BTC to Alice'])

# Display the complete chain
blockchain.display_chain()

# Validate the chain
is_valid = blockchain.validate_chain()
print(f"\n🔍 Blockchain is {'valid ✅' if is_valid else 'invalid ❌'}")

🔗 Creating a Simple Blockchain
Genesis block created with hash: 1e52246a1dd9d09a...
⛏️  Mining new block with 1 transactions...
✅ Block mined! Nonce: 386152, Time: 2.23s
   Hash: 00005fa80d7144f60ad557c5ff6e6e4ee55ec470611aeadc9d57dad31e043edf
⛏️  Mining new block with 2 transactions...
✅ Block mined! Nonce: 101646, Time: 0.62s
   Hash: 0000714b753260650c57d8746188895f7921f7e86685912a23f051dc6abddcdf
⛏️  Mining new block with 1 transactions...
✅ Block mined! Nonce: 6158, Time: 0.04s
   Hash: 0000c0c0148f97cc3e0fb26ee181de6b2b2fcb54b6b9ed6652aab8cf3d4a85fe

📊 Blockchain Status:
Block #0:
  Timestamp: Sun Jul  6 22:14:25 2025
  Transactions: ['Genesis Block']
  Previous Hash: 0...
  Nonce: 0
  Hash: 1e52246a1dd9d09a...
  Valid: ✅
----------------------------------------
Block #1:
  Timestamp: Sun Jul  6 22:14:25 2025
  Transactions: ['Alice sends 5 BTC to Bob']
  Previous Hash: 1e52246a1dd9d09a...
  Nonce: 386152
  Hash: 00005fa80d7144f6...
  Valid: ✅
------------------------------------

## Page 2: Transactions

### How a Transaction Works

1.  **What is a Coin?**
    A "coin" is defined as a **chain of digital signatures**. It's like the history of endorsements on a paper check.

2.  **Transferring Ownership:**
    To send a coin to Alice, the current owner signs a hash of the previous transaction and Alice's public key. This new transaction is added to the end of the coin's chain.

3.  **Verification:**
    Anyone can check the chain of signatures to verify the coin's legitimate history.

### The Big Problem: Double-Spending

Since it's all digital, what stops the owner from sending the same coin to two different people (e.g., Alice and Bob)? This is the **double-spending problem**.

### The Solution (Introduction)

To prevent double-spending, the network needs a way to agree on a **single, shared history of transactions**. The paper proposes:

1.  All transactions must be **publicly announced**.
2.  The network needs a system to agree on the **order** in which transactions were received.

This sets the stage for the Timestamp Server and Proof-of-Work.

In [4]:
# Digital Signatures and Double-Spending Demonstration
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
import base64

class DigitalCoin:
    def __init__(self):
        self.transaction_chain = []
        
    def generate_keypair(self):
        """Generate a public-private key pair"""
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048
        )
        public_key = private_key.public_key()
        return private_key, public_key
    
    def sign_transaction(self, transaction_data, private_key):
        """Sign a transaction with a private key"""
        signature = private_key.sign(
            transaction_data.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode()
    
    def verify_signature(self, transaction_data, signature, public_key):
        """Verify a transaction signature"""
        try:
            public_key.verify(
                base64.b64decode(signature),
                transaction_data.encode(),
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True
        except:
            return False
    
    def create_transaction(self, previous_tx_hash, sender_private_key, recipient_public_key):
        """Create a new transaction in the coin's chain"""
        # Get public key for sender
        sender_public_key = sender_private_key.public_key()
        
        # Create transaction data
        transaction_data = f"Previous: {previous_tx_hash}, To: {recipient_public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode()[:50]}..."
        
        # Sign the transaction
        signature = self.sign_transaction(transaction_data, sender_private_key)
        
        transaction = {
            'previous_hash': previous_tx_hash,
            'sender_public_key': sender_public_key,
            'recipient_public_key': recipient_public_key,
            'transaction_data': transaction_data,
            'signature': signature,
            'timestamp': time.time()
        }
        
        self.transaction_chain.append(transaction)
        return transaction
    
    def verify_coin_history(self):
        """Verify all signatures in the coin's transaction chain"""
        for i, tx in enumerate(self.transaction_chain):
            if not self.verify_signature(tx['transaction_data'], tx['signature'], tx['sender_public_key']):
                print(f"❌ Invalid signature at transaction {i}")
                return False
        print("✅ All signatures in coin history are valid")
        return True

# Demonstrate digital signatures and coin transfers
print("🪙 Digital Coin Demonstration")
print("=" * 50)

# Create keypairs for Alice, Bob, and Charlie
alice_private, alice_public = DigitalCoin().generate_keypair()
bob_private, bob_public = DigitalCoin().generate_keypair()
charlie_private, charlie_public = DigitalCoin().generate_keypair()

print("👤 Generated keypairs for Alice, Bob, and Charlie")

# Create a coin and transfer it
coin = DigitalCoin()

# Genesis transaction (coin creation)
genesis_tx = coin.create_transaction("genesis", alice_private, alice_public)
print("🆕 Created genesis transaction (Alice owns the coin)")

# Alice sends coin to Bob
alice_to_bob = coin.create_transaction(
    hashlib.sha256(str(genesis_tx).encode()).hexdigest(),
    alice_private, 
    bob_public
)
print("💸 Alice transferred coin to Bob")

# Bob sends coin to Charlie
bob_to_charlie = coin.create_transaction(
    hashlib.sha256(str(alice_to_bob).encode()).hexdigest(),
    bob_private,
    charlie_public
)
print("💸 Bob transferred coin to Charlie")

# Verify the entire coin history
print("\n🔍 Verifying coin transaction history:")
coin.verify_coin_history()

print(f"\n📊 Coin has {len(coin.transaction_chain)} transactions in its history")

# Demonstrate double-spending attempt
print("\n🚨 Demonstrating Double-Spending Problem:")
print("What if Alice tries to send the same coin to David while Bob still owns it?")

david_private, david_public = DigitalCoin().generate_keypair()
try:
    # Alice attempts to double-spend (this would fail in a real system)
    double_spend_attempt = {
        'previous_hash': hashlib.sha256(str(genesis_tx).encode()).hexdigest(),
        'from': 'Alice',
        'to': 'David',
        'note': 'This is a double-spending attempt!'
    }
    print("⚠️  Alice created a conflicting transaction")
    print("💡 In Bitcoin, this is prevented by the network consensus mechanism")
    print("   Only one version of the transaction history can be valid")
except Exception as e:
    print(f"❌ Double-spending prevented: {e}")

print("\n🔑 Key Insight: Digital signatures prove ownership, but we still need")
print("   a mechanism to prevent double-spending. That's where the blockchain comes in!")

ModuleNotFoundError: No module named 'cryptography'

## Page 3: Timestamp Server

### The Core Idea

The solution to the double-spending problem starts with a **timestamp server**. Its purpose is to create a decentralized, agreed-upon history of transactions.

### How it Works

1.  **Hashing a Block:** A block of transactions is collected, and a **hash** (a unique digital fingerprint) of the entire block is calculated.

2.  **Public Broadcasting:** This hash is then widely published, proving that the data in the block existed at that specific time.

### Creating the Chain

This is the key concept of the blockchain:

*   Each new timestamp **must include the previous timestamp's hash** in its own data before being hashed.
*   This creates a **chain**, where each new link cryptographically depends on the one before it.

This structure makes the history incredibly secure. To change an old block, an attacker would have to redo the work for that block and all subsequent blocks in the chain.

## Section 4: Proof-of-Work

*   **The Goal:** To implement the timestamp server in a peer-to-peer way, requiring the hash of a block to start with a certain number of zero bits.
*   **The Method:** Miners must repeatedly hash the block header with a different "nonce" until they find a hash that meets the difficulty target. This is what `assignment3.js` simulates.
*   **Difficulty Adjustment:** The difficulty is automatically adjusted by the network to keep the average time between new blocks at around 10 minutes.

## Section 5: The Network

The step-by-step process of how the network operates:
1.  New transactions are broadcast to all nodes.
2.  Each node collects new transactions into a block.
3.  Each node works on finding the difficult proof-of-work for its block.
4.  When a node finds a proof-of-work, it broadcasts the block to all other nodes.
5.  Nodes only accept the block if all transactions in it are valid and not already spent.
6.  Nodes show they accept the block by starting to work on the *next* block, using the hash of the accepted block as the previous hash. The longest chain is always considered the correct one.

## Section 6: Incentive

*   **Why Participate?**
    1.  **Block Reward:** The creator of a block gets a reward of new coins.
    2.  **Transaction Fees:** The creator also gets to keep any transaction fees from the transactions within that block.
*   **The Result:** This encourages honest participation and secures the network.

## Section 7: Reclaiming Disk Space

*   **The Problem:** Storing every transaction forever would make the blockchain enormous.
*   **The Solution (Merkle Trees):** Once transactions are buried deep enough, they can be discarded. Only the block header, containing the **root hash** of a Merkle Tree of all the block's transactions, needs to be kept.

## Section 8: Simplified Payment Verification (SPV)

*   **The Concept:** It's possible to verify payments without running a full network node.
*   **How it Works:** An SPV client downloads only the **block headers**. It can then ask the network for the Merkle branch that links a specific transaction to its place in a block, providing cryptographic proof of its existence.

## Section 9: Combining and Splitting Value

*   **Combining:** A transaction can have multiple inputs to combine smaller amounts.
*   **Splitting:** A transaction typically has two outputs: one for the payment and one that sends the "change" back to the sender using a new address.

## Section 10: Privacy

*   **How it Works:** The network is **pseudonymous**, not anonymous. Public keys are not directly linked to real-world identities.
*   **The Catch:** All transactions are public. If a key is linked to a person, all their transactions can be traced.
*   **The Recommendation:** Use a new key pair for every transaction.

## Section 11: Calculations

This section provides a statistical analysis showing that the probability of an attacker successfully creating an alternative, longer chain decreases exponentially as more blocks are added to the honest chain.

## Section 12: Conclusion

A summary of the system: a trustless, peer-to-peer electronic cash system that is secure as long as honest nodes control the majority of CPU power.

In [5]:
# Interactive Proof-of-Work Demonstration
import matplotlib.pyplot as plt
import numpy as np

class ProofOfWorkDemo:
    def __init__(self, difficulty=4):
        self.difficulty = difficulty
        self.target = '0' * difficulty
        
    def hash_with_nonce(self, data, nonce):
        """Hash data with a nonce value"""
        combined = f"{data}{nonce}"
        return hashlib.sha256(combined.encode()).hexdigest()
    
    def mine_block(self, block_data, show_attempts=False):
        """Demonstrate the mining process"""
        print(f"⛏️  Mining block with difficulty {self.difficulty} (target: {self.target}...)")
        print(f"📦 Block data: {block_data}")
        
        nonce = 0
        attempts = []
        start_time = time.time()
        
        while True:
            hash_result = self.hash_with_nonce(block_data, nonce)
            attempts.append((nonce, hash_result))
            
            if show_attempts and nonce < 10:
                print(f"   Attempt {nonce}: nonce={nonce} → {hash_result}")
            
            if hash_result.startswith(self.target):
                mining_time = time.time() - start_time
                print(f"✅ Success! Found valid hash after {nonce + 1} attempts")
                print(f"   Winning nonce: {nonce}")
                print(f"   Winning hash: {hash_result}")
                print(f"   Mining time: {mining_time:.3f} seconds")
                return nonce, hash_result, attempts
            
            nonce += 1
            
            # Safety check for demo
            if nonce > 100000:
                print("⚠️  Taking too long, stopping demo")
                break
                
        return None, None, attempts
    
    def demonstrate_difficulty_scaling(self):
        """Show how difficulty affects mining time"""
        block_data = "Sample block with transactions"
        difficulties = [1, 2, 3, 4]
        mining_times = []
        attempt_counts = []
        
        print("\n📈 Difficulty Scaling Demonstration:")
        print("=" * 50)
        
        for diff in difficulties:
            self.difficulty = diff
            self.target = '0' * diff
            
            start_time = time.time()
            nonce, hash_result, attempts = self.mine_block(block_data)
            mining_time = time.time() - start_time
            
            if nonce is not None:
                mining_times.append(mining_time)
                attempt_counts.append(len(attempts))
                print(f"Difficulty {diff}: {len(attempts)} attempts, {mining_time:.3f}s")
            else:
                mining_times.append(None)
                attempt_counts.append(None)
            print("-" * 30)
        
        return difficulties, mining_times, attempt_counts
    
    def visualize_hash_distribution(self, sample_size=1000):
        """Visualize the distribution of hash attempts"""
        data = "sample_block_data"
        hash_values = []
        
        print(f"\n📊 Analyzing {sample_size} hash attempts...")
        
        for nonce in range(sample_size):
            hash_result = self.hash_with_nonce(data, nonce)
            # Convert first 8 characters of hash to integer for visualization
            hash_int = int(hash_result[:8], 16)
            hash_values.append(hash_int)
        
        # Create histogram
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.hist(hash_values, bins=50, alpha=0.7, color='blue', edgecolor='black')
        plt.title('Distribution of Hash Values')
        plt.xlabel('Hash Value (first 8 hex chars as int)')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        
        # Show leading zeros analysis
        leading_zeros = []
        for nonce in range(sample_size):
            hash_result = self.hash_with_nonce(data, nonce)
            zeros = 0
            for char in hash_result:
                if char == '0':
                    zeros += 1
                else:
                    break
            leading_zeros.append(zeros)
        
        plt.subplot(1, 2, 2)
        max_zeros = max(leading_zeros)
        bins = range(max_zeros + 2)
        plt.hist(leading_zeros, bins=bins, alpha=0.7, color='green', edgecolor='black')
        plt.title('Leading Zeros Distribution')
        plt.xlabel('Number of Leading Zeros')
        plt.ylabel('Frequency')
        plt.grid(True, alpha=0.3)
        
        # Add difficulty lines
        for diff in range(1, min(6, max_zeros + 1)):
            expected_prob = (1/16) ** diff
            expected_count = sample_size * expected_prob
            plt.axvline(x=diff-0.5, color='red', linestyle='--', alpha=0.7)
            plt.text(diff, expected_count, f'Diff {diff}', rotation=90, alpha=0.7)
        
        plt.tight_layout()
        plt.show()
        
        return leading_zeros

# Run proof-of-work demonstrations
pow_demo = ProofOfWorkDemo(difficulty=3)

# Single mining demonstration
print("🎯 Single Block Mining Demo:")
nonce, hash_result, attempts = pow_demo.mine_block("Block #123: Alice->Bob: 5 BTC", show_attempts=True)

# Difficulty scaling demonstration
difficulties, times, counts = pow_demo.demonstrate_difficulty_scaling()

# Hash distribution visualization
print("\n🔍 Hash Analysis:")
leading_zeros_data = pow_demo.visualize_hash_distribution(1000)

print(f"\n📈 Statistics from {len(leading_zeros_data)} hash attempts:")
print(f"   Max leading zeros found: {max(leading_zeros_data)}")
print(f"   Average leading zeros: {np.mean(leading_zeros_data):.2f}")
print(f"   Hashes with 3+ leading zeros: {sum(1 for x in leading_zeros_data if x >= 3)}")
print(f"   Probability of 3+ zeros: {sum(1 for x in leading_zeros_data if x >= 3)/len(leading_zeros_data)*100:.2f}%")

ModuleNotFoundError: No module named 'matplotlib'

In [6]:
# Complete Bitcoin Network Simulation
class BitcoinNetworkSimulation:
    def __init__(self, num_nodes=10):
        self.nodes = {}
        self.network_blocks = []
        self.mempool = []  # Pending transactions
        self.difficulty = 4
        self.block_reward = 6.25  # Current Bitcoin block reward
        
        # Initialize nodes
        for i in range(num_nodes):
            self.nodes[f"node_{i}"] = {
                'id': f"node_{i}",
                'balance': random.uniform(0, 100),
                'is_miner': random.random() < 0.3,  # 30% are miners
                'hash_power': random.uniform(1, 10) if random.random() < 0.3 else 0,
                'blocks_mined': 0,
                'transactions_created': 0
            }
    
    def create_transaction(self, sender, receiver, amount):
        """Create a new transaction"""
        if sender not in self.nodes or receiver not in self.nodes:
            return None
            
        if self.nodes[sender]['balance'] < amount:
            print(f"❌ Insufficient funds: {sender} has {self.nodes[sender]['balance']:.2f}, needs {amount}")
            return None
        
        tx = {
            'id': hashlib.sha256(f"{sender}{receiver}{amount}{time.time()}".encode()).hexdigest()[:16],
            'sender': sender,
            'receiver': receiver,
            'amount': amount,
            'timestamp': time.time(),
            'fee': amount * 0.001  # 0.1% transaction fee
        }
        
        self.mempool.append(tx)
        print(f"💸 Transaction created: {sender} → {receiver}: {amount} BTC")
        return tx
    
    def mine_block(self, miner_id):
        """Simulate mining a block"""
        if not self.mempool:
            print("📭 No transactions to mine")
            return None
        
        # Select transactions from mempool
        selected_txs = self.mempool[:5]  # Take up to 5 transactions
        total_fees = sum(tx['fee'] for tx in selected_txs)
        
        # Create block
        block = {
            'index': len(self.network_blocks),
            'miner': miner_id,
            'transactions': selected_txs,
            'timestamp': time.time(),
            'previous_hash': self.network_blocks[-1]['hash'] if self.network_blocks else '0',
            'merkle_root': self.calculate_merkle_root(selected_txs),
            'difficulty': self.difficulty,
            'nonce': 0,
            'hash': None,
            'block_reward': self.block_reward,
            'total_fees': total_fees
        }
        
        # Simulate proof-of-work (simplified)
        miner_hash_power = self.nodes[miner_id]['hash_power']
        mining_time = random.exponential(10 / miner_hash_power)  # Higher hash power = faster mining
        
        time.sleep(min(0.1, mining_time))  # Brief pause for demo
        
        # Find valid hash
        while True:
            block_string = f"{block['index']}{block['previous_hash']}{block['merkle_root']}{block['timestamp']}{block['nonce']}"
            block['hash'] = hashlib.sha256(block_string.encode()).hexdigest()
            
            if block['hash'].startswith('0' * self.difficulty):
                break
            block['nonce'] += 1
            
            if block['nonce'] > 100000:  # Safety check
                self.difficulty = max(1, self.difficulty - 1)
                block['nonce'] = 0
        
        # Add block to chain
        self.network_blocks.append(block)
        
        # Update balances
        for tx in selected_txs:
            self.nodes[tx['sender']]['balance'] -= (tx['amount'] + tx['fee'])
            self.nodes[tx['receiver']]['balance'] += tx['amount']
            self.nodes[tx['sender']]['transactions_created'] += 1
        
        # Reward miner
        self.nodes[miner_id]['balance'] += (self.block_reward + total_fees)
        self.nodes[miner_id]['blocks_mined'] += 1
        
        # Remove mined transactions from mempool
        for tx in selected_txs:
            if tx in self.mempool:
                self.mempool.remove(tx)
        
        print(f"⛏️  Block #{block['index']} mined by {miner_id}")
        print(f"   Transactions: {len(selected_txs)}, Reward: {self.block_reward + total_fees:.4f} BTC")
        print(f"   Hash: {block['hash'][:20]}...")
        
        return block
    
    def calculate_merkle_root(self, transactions):
        """Calculate Merkle root of transactions (simplified)"""
        if not transactions:
            return hashlib.sha256(b'').hexdigest()
        
        tx_hashes = [hashlib.sha256(str(tx).encode()).hexdigest() for tx in transactions]
        
        while len(tx_hashes) > 1:
            if len(tx_hashes) % 2 == 1:
                tx_hashes.append(tx_hashes[-1])  # Duplicate last hash if odd number
            
            new_level = []
            for i in range(0, len(tx_hashes), 2):
                combined = tx_hashes[i] + tx_hashes[i + 1]
                new_level.append(hashlib.sha256(combined.encode()).hexdigest())
            
            tx_hashes = new_level
        
        return tx_hashes[0] if tx_hashes else hashlib.sha256(b'').hexdigest()
    
    def simulate_network_activity(self, num_transactions=20, num_blocks=5):
        """Simulate network activity"""
        print("🌐 Simulating Bitcoin Network Activity")
        print("=" * 60)
        
        # Generate random transactions
        node_ids = list(self.nodes.keys())
        for _ in range(num_transactions):
            sender = random.choice(node_ids)
            receiver = random.choice([n for n in node_ids if n != sender])
            amount = random.uniform(0.1, 5.0)
            self.create_transaction(sender, receiver, amount)
        
        print(f"\n📝 Created {len(self.mempool)} transactions in mempool")
        
        # Mine blocks
        miners = [node_id for node_id, node in self.nodes.items() if node['is_miner']]
        
        for i in range(num_blocks):
            if not self.mempool:
                break
                
            # Select miner based on hash power (weighted random)
            if miners:
                weights = [self.nodes[miner]['hash_power'] for miner in miners]
                selected_miner = random.choices(miners, weights=weights)[0]
                self.mine_block(selected_miner)
                print()
        
        # Display network statistics
        self.display_network_stats()
    
    def display_network_stats(self):
        """Display comprehensive network statistics"""
        print("📊 Network Statistics")
        print("=" * 60)
        
        # Blockchain stats
        print(f"📦 Blockchain: {len(self.network_blocks)} blocks")
        print(f"⏳ Pending transactions: {len(self.mempool)}")
        print(f"⚡ Current difficulty: {self.difficulty}")
        
        # Node statistics
        miners = [n for n in self.nodes.values() if n['is_miner']]
        total_hash_power = sum(n['hash_power'] for n in miners)
        
        print(f"\n👥 Network Nodes:")
        print(f"   Total nodes: {len(self.nodes)}")
        print(f"   Miners: {len(miners)}")
        print(f"   Total hash power: {total_hash_power:.2f}")
        
        # Top miners
        top_miners = sorted(miners, key=lambda x: x['blocks_mined'], reverse=True)[:5]
        print(f"\n🏆 Top Miners:")
        for i, miner in enumerate(top_miners):
            hash_share = (miner['hash_power'] / total_hash_power) * 100 if total_hash_power > 0 else 0
            print(f"   {i+1}. {miner['id']}: {miner['blocks_mined']} blocks, {hash_share:.1f}% hash power")
        
        # Wealth distribution
        sorted_nodes = sorted(self.nodes.values(), key=lambda x: x['balance'], reverse=True)
        total_wealth = sum(n['balance'] for n in self.nodes.values())
        
        print(f"\n💰 Wealth Distribution:")
        print(f"   Total network wealth: {total_wealth:.2f} BTC")
        print(f"   Top 5 wealthiest nodes:")
        for i, node in enumerate(sorted_nodes[:5]):
            wealth_share = (node['balance'] / total_wealth) * 100 if total_wealth > 0 else 0
            print(f"   {i+1}. {node['id']}: {node['balance']:.2f} BTC ({wealth_share:.1f}%)")
        
        # Transaction statistics
        total_transactions = sum(len(block['transactions']) for block in self.network_blocks)
        total_fees = sum(sum(tx['fee'] for tx in block['transactions']) for block in self.network_blocks)
        
        print(f"\n💸 Transaction Statistics:")
        print(f"   Total transactions processed: {total_transactions}")
        print(f"   Total fees collected: {total_fees:.4f} BTC")
        print(f"   Average transactions per block: {total_transactions/len(self.network_blocks) if self.network_blocks else 0:.1f}")
        
        return {
            'blocks': len(self.network_blocks),
            'pending_txs': len(self.mempool),
            'total_nodes': len(self.nodes),
            'miners': len(miners),
            'total_hash_power': total_hash_power,
            'total_wealth': total_wealth,
            'total_transactions': total_transactions,
            'total_fees': total_fees
        }

# Run the complete simulation
print("🚀 Running Complete Bitcoin Network Simulation")
print("=" * 70)

# Create and run simulation
network = BitcoinNetworkSimulation(num_nodes=15)
network.simulate_network_activity(num_transactions=25, num_blocks=8)

print("\n🎓 Simulation Complete!")
print("This demonstrates the key concepts from the Bitcoin whitepaper:")
print("✅ Peer-to-peer transactions without trusted third parties")
print("✅ Cryptographic proof through digital signatures")
print("✅ Proof-of-work consensus mechanism")
print("✅ Blockchain for transaction ordering and double-spend prevention")
print("✅ Economic incentives for network security")
print("✅ Decentralized network of nodes")

🚀 Running Complete Bitcoin Network Simulation
🌐 Simulating Bitcoin Network Activity
💸 Transaction created: node_8 → node_2: 1.7577461921966888 BTC
💸 Transaction created: node_5 → node_9: 1.2370299308424078 BTC
💸 Transaction created: node_9 → node_10: 3.2777000781919776 BTC
💸 Transaction created: node_8 → node_0: 1.2793827662950699 BTC
💸 Transaction created: node_4 → node_2: 0.6310299760951961 BTC
💸 Transaction created: node_2 → node_14: 2.7635360957176855 BTC
💸 Transaction created: node_6 → node_2: 2.8312841198293905 BTC
💸 Transaction created: node_13 → node_3: 1.1618106969260442 BTC
💸 Transaction created: node_4 → node_11: 3.0181720399375265 BTC
💸 Transaction created: node_12 → node_0: 0.27536071955363584 BTC
💸 Transaction created: node_1 → node_4: 3.578742414564667 BTC
💸 Transaction created: node_13 → node_14: 3.8766811401034627 BTC
💸 Transaction created: node_1 → node_3: 2.564366985798744 BTC
💸 Transaction created: node_13 → node_8: 4.276829514212777 BTC
💸 Transaction created: node_

AttributeError: module 'random' has no attribute 'exponential'

# 🎯 Key Takeaways and Next Steps

## 🧠 What We've Learned

This notebook has provided a comprehensive analysis of the Bitcoin whitepaper with interactive demonstrations of:

### Core Concepts
1. **Trust vs. Cryptographic Proof** - Moving from trust-based systems to mathematical certainty
2. **Digital Signatures** - How ownership is proven and transferred
3. **Double-Spending Prevention** - The fundamental problem Bitcoin solves
4. **Proof-of-Work** - The consensus mechanism that secures the network
5. **Blockchain Structure** - How transactions are linked and secured over time

### Technical Implementation
- ✅ **Cryptographic Hashing** - SHA-256 and hash-based security
- ✅ **Digital Signatures** - RSA key pairs and transaction signing
- ✅ **Mining Simulation** - Proof-of-work difficulty and nonce finding
- ✅ **Network Consensus** - How nodes agree on transaction history
- ✅ **Economic Incentives** - Block rewards and transaction fees

## 🔮 Impact and Legacy

The Bitcoin whitepaper introduced concepts that revolutionized:

- **Digital Money** - First successful decentralized digital currency
- **Blockchain Technology** - Now used across industries
- **Smart Contracts** - Enabled by Bitcoin's scripting system
- **DeFi (Decentralized Finance)** - Building on Bitcoin's foundation
- **Web3** - Decentralized internet applications

## 📈 Next Learning Steps

### Immediate Next Steps
1. **Run the Code** - Execute all the interactive examples in this notebook
2. **Experiment** - Modify parameters (difficulty, network size, etc.)
3. **Visualize** - Study the graphs and distributions generated

### Advanced Learning Path
1. **Bitcoin Development** - Learn Bitcoin Core development
2. **Lightning Network** - Understand layer-2 scaling solutions
3. **Ethereum** - Explore smart contracts and DApps
4. **Other Cryptocurrencies** - Study alternative consensus mechanisms
5. **Blockchain Development** - Build your own blockchain applications

### Recommended Resources
- [Bitcoin Developer Documentation](https://developer.bitcoin.org/)
- [Mastering Bitcoin by Andreas Antonopoulos](https://github.com/bitcoinbook/bitcoinbook)
- [Bitcoin Improvement Proposals (BIPs)](https://github.com/bitcoin/bips)
- [Bitcoin Core Source Code](https://github.com/bitcoin/bitcoin)

## 🛠️ Practical Applications

With this understanding, you can now:

- **Analyze** other cryptocurrency whitepapers
- **Understand** blockchain news and developments
- **Evaluate** new cryptocurrency projects
- **Contribute** to open-source blockchain projects
- **Build** blockchain applications

## 🌟 Final Thoughts

Satoshi Nakamoto's 9-page whitepaper solved the Byzantine Generals Problem and created the foundation for a new digital economy. The elegance of the solution lies in its simplicity:

> **"The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power."**

This simple principle enables trustless, decentralized digital money - one of the most significant technological innovations of the 21st century.

---

**🎓 Congratulations! You now understand the fundamental principles that power Bitcoin and the entire cryptocurrency ecosystem.**